In [1]:
#!pip install selenium
#!pip install scrapy

In [2]:
import numpy as np
import pandas as pd
from scrapy.selector import Selector
from selenium import webdriver 
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore")
import random

In [3]:
driver = webdriver.Chrome(r'D:\UMD Master Year 2 College Work\Fall 2022-2023\INST737-IM01\chromedriver')
url = 'https://www.imdb.com/title/tt0086190/reviews?ref_=tt_ov_rt' 
randomNumber = random.randint(0,4)
time.sleep(1)
driver.get(url)
time.sleep(1)
print(driver.title)
time.sleep(1)
body = driver.find_element(By.CSS_SELECTOR, 'body')
body.send_keys(Keys.END)
time.sleep(randomNumber)
# body.send_keys(Keys.PAGE_DOWN)
# time.sleep(1)
# body.send_keys(Keys.PAGE_DOWN)
# time.sleep(1)
# body.send_keys(Keys.PAGE_DOWN)

Star Wars: Episode VI - Return of the Jedi (1983) - Star Wars: Episode VI - Return of the Jedi (1983) - User Reviews - IMDb


In [4]:
sel = Selector(text = driver.page_source)
review_counts = sel.css('.lister .header span::text').extract_first().replace(',','').split(' ')[0]
more_review_pages = int(int(review_counts)/25)

In [5]:
for i in tqdm(range(more_review_pages)):
    try:
        css_selector = 'load-more-trigger'
        driver.find_element(By.ID, css_selector).click()
        body.send_keys(Keys.END)
        time.sleep(randomNumber)
    except:
        pass

100%|██████████| 38/38 [01:23<00:00,  2.20s/it]


In [6]:
reviews = driver.find_elements(By.CSS_SELECTOR, 'div.review-container')
first_review = reviews[0]
sel2 = Selector(text = first_review.get_attribute('innerHTML'))
rating = sel2.css('.rating-other-user-rating span::text').extract_first().strip()

In [7]:
review = sel2.css('.text.show-more__control::text').extract_first().strip()
review_date = sel2.css('.review-date::text').extract_first().strip()
author = sel2.css('.display-name-link a::text').extract_first().strip()
review_title = sel2.css('a.title::text').extract_first().strip()
review_url = sel2.css('a.title::attr(href)').extract_first().strip()
#helpfulness = sel2.css('.actions.text-muted::text').extract_first().strip()
print('nRating:',rating)
print('nreview_title:',review_title)
print('nAuthor:',author)
print('nreview_date:',review_date)
print('nreview:',review)
#print('nhelpfulness:',helpfulness)

nRating: 10
nreview_title: The most peculiar Star Wars and under appreciated
nAuthor: UniqueParticle
nreview_date: 16 January 2020
nreview: Totally should've won at least some of the awards it was nominated for! A very different adventure story with lots of adorable characters, more truths and the epic Emperor Palpatine! I love all the majestic forest aspects. Still crazy to me that they filmed so many parts especially the chases. The bulk of this art in cinema is done so well and always entertaining. Quite lovable many years later and not many movies have that kind of force!


In [8]:
rating_list = []
review_date_list = []
review_title_list = []
author_list = []
review_list = []
review_url_list = []
error_url_list = []
error_msg_list = []
reviews = driver.find_elements(By.CSS_SELECTOR, 'div.review-container')

for d in tqdm(reviews):
    try:
        sel2 = Selector(text = d.get_attribute('innerHTML'))
        try:
            rating = sel2.css('.rating-other-user-rating span::text').extract_first()
        except:
            rating = np.NaN
        try:
            review = sel2.css('.text.show-more__control::text').extract_first()
        except:
            review = np.NaN
        try:
            review_date = sel2.css('.review-date::text').extract_first()
        except:
            review_date = np.NaN    
        try:
            author = sel2.css('.display-name-link a::text').extract_first()
        except:
            author = np.NaN    
        try:
            review_title = sel2.css('a.title::text').extract_first()
        except:
            review_title = np.NaN
        try:
            review_url = sel2.css('a.title::attr(href)').extract_first()
        except:
            review_url = np.NaN
        rating_list.append(rating)
        review_date_list.append(review_date)
        review_title_list.append(review_title)
        author_list.append(author)
        review_list.append(review)
        review_url_list.append(review_url)
    except Exception as e:
        error_url_list.append(url)
        error_msg_list.append(e)
review_df = pd.DataFrame({
    'Review_Date':review_date_list,
    'Author':author_list,
    'Rating':rating_list,
    'Review_Title':review_title_list,
    'Review':review_list,
    'Review_Url':review_url
    })

100%|██████████| 940/940 [00:24<00:00, 38.99it/s]


In [9]:
review_df

,Review_Date,Author,Rating,Review_Title,Review,Review_Url
0,16 January 2020,UniqueParticle,10,The most peculiar Star Wars and under appreci...,Totally should've won at least some of the awa...,/review/rw4680668/?ref_=tt_urv
1,2 December 2005,evanston_dad,10,"The Boy Who Loved ""Star Wars""\n","A long time ago, in a galaxy far, far away.......",/review/rw4680668/?ref_=tt_urv
2,12 February 2006,ccthemovieman-1,9,One Of The Best In The Series\n,Up until the sixth and last episode of the Sta...,/review/rw4680668/?ref_=tt_urv
3,30 March 2020,bevo-13678,10,Good\n,I like the bit when the ewoks crush an ATST wi...,/review/rw4680668/?ref_=tt_urv
4,28 April 2016,ivo-cobra8,8,"Flawed, very entertaining solid third sequel ...",Star Wars: Episode VI – Return of the Jedi (19...,/review/rw4680668/?ref_=tt_urv
...,...,...,...,...,...,...
935,19 August 2016,lanamay-03293,8,A Great Film\n,I was never into the star wars films so i neve...,/review/rw4680668/?ref_=tt_urv
936,2 August 2022,kghijselinck,10,Tribute to John Williams\n,A couple of weeks ago I had (again) the opport...,/review/rw4680668/?ref_=tt_urv
937,1 April 2019,mrklein-84525,9,Family values\n,As a cub viewing the scene where father saves ...,/review/rw4680668/?ref_=tt_urv
938,10 September 2006,dolius,10,Good Ending And Good Movie\n,As Most Off You Might off Seen Star Wars: Retu...,/review/rw4680668/?ref_=tt_urv


In [10]:
#953 reviews

In [13]:
#review_df.to_csv(r'D:\UMD Master Year 2 College Work\Fall 2022-2023\INST737-IM01\INST737_Star_Wars_IMDB_StarWars6.csv')